In [ ]:
# import sys

In [ ]:
# !{sys.executable} -m pip install pandas

In [1]:
import findspark

In [2]:
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')

In [3]:
from pyspark import *
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.feature import CountVectorizer, StopWordsRemover,Word2Vec
from pyspark.ml.feature import Tokenizer, RegexTokenizer,HashingTF,IDF
from pyspark.ml.classification import LogisticRegression,LinearSVC
from pyspark.mllib.classification import LogisticRegressionWithLBFGS

from pyspark.mllib.util import MLUtils
from pyspark.mllib.evaluation import MultilabelMetrics


In [4]:
import pandas as pd

In [5]:
import os
java_home = os.environ.get('JAVA_HOME', None)
os.environ['SPARK_LOCAL_IP'] = 'localhost'
print(java_home)
print(os.environ.get('SPARK_LOCAL_IP', None))

None
localhost


In [6]:
# spark_conf = SparkConf().setAppName("test")
# spark = SparkContext(conf = spark_conf)
# SparkConf().getAll()

In [7]:
spark = SparkSession.builder.master('local[*]')\
                        .config("spark.driver.memory","15g")\
                        .getOrCreate()

In [8]:
feature_count=35000

In [9]:
df=pd.read_csv("train.csv")
myschema=StructType([StructField("movie_id",StringType(),True)\
                     ,StructField("movie_name",StringType(),True)\
                     ,StructField("plot",StringType(),True)\
                     ,StructField("genre",StringType(),True)\
                    ])
spark_df=spark.createDataFrame(df,schema=myschema)
df=spark_df
mapping_df = spark.read.load("mapping.csv", format='csv', sep=',', inferSchema=True, header=True)
for row in mapping_df.rdd.collect():
    val=row['0']
    def create_cols(row):
        if val in row:
            return 1
        else:
            return 0
    udfconvert=udf(create_cols,IntegerType())
    df=df.withColumn(val,udfconvert("genre"))

In [10]:
tokenizer = Tokenizer(inputCol="plot", outputCol="words")

regexTokenizer = RegexTokenizer(inputCol="plot", outputCol="candidate_words", pattern="\\W")
# alternatively, pattern="\\w+", gaps(False)

countTokens = udf(lambda words: len(words), IntegerType())


regexTokenized = regexTokenizer.transform(df)
# regexTokenized.select("plot", "candidate_") \
#     .withColumn("tokens", countTokens(col("words"))).show(truncate=True)
# remover = StopWordsRemover(inputCol="words", outputCol="candidate_words")
filtered_df = regexTokenized


In [15]:
test_df=pd.read_csv("test.csv")
testschema=StructType([StructField("movie_id",StringType(),True)\
                     ,StructField("movie_name",StringType(),True)\
                     ,StructField("plot",StringType(),True)\
                    ])
test_spark_df=spark.createDataFrame(test_df,schema=testschema)

test_regexTokenized = regexTokenizer.transform(test_spark_df)
test_regexTokenized.select("plot", "candidate_words") \
    .withColumn("tokens", countTokens(col("candidate_words"))).show(truncate=True)

test_filtered_df = test_regexTokenized

#test_model = cv.fit(test_filtered_df)



+--------------------+--------------------+------+
|                plot|     candidate_words|tokens|
+--------------------+--------------------+------+
|The film is based...|[the, film, is, b...|   786|
|A group of teenag...|[a, group, of, te...|    52|
|This story of a Z...|[this, story, of,...|   396|
|The Stooges play ...|[the, stooges, pl...|   116|
|A soldier-of-fort...|[a, soldier, of, ...|    35|
|Set in the Northw...|[set, in, the, no...|   407|
|Like in many othe...|[like, in, many, ...|   328|
|Mickey and the Sc...|[mickey, and, the...|    27|
|In the desert wil...|[in, the, desert,...|   396|
|Bimbo and Koko ar...|[bimbo, and, koko...|    54|
|Tahaan  lives wit...|[tahaan, lives, w...|   372|
|Betty is startled...|[betty, is, start...|   167|
|Nirmal ([[Karthik...|[nirmal, karthik,...|   180|
|A group of journa...|[a, group, of, jo...|   426|
|Vaibhavari Sahay,...|[vaibhavari, saha...|   330|
|In 1947, the movi...|[in, 1947, the, m...|   391|
|Ninja Resurrectio...|[ninja, r

In [ ]:
#######Part 1 Creating Term-document Matrix ########

In [16]:
word2vec=Word2Vec(inputCol="candidate_words",outputCol="w2v")

In [17]:
wtv_model=word2vec.fit(filtered_df)
wtv=wtv_model.transform(filtered_df)

test_wtv_model=word2vec.fit(test_filtered_df)
test_wtv=test_wtv_model.transform(test_filtered_df)

In [18]:
result=wtv
test_result=test_wtv
log_result=test_result
for row in mapping_df.rdd.collect():
    val=row['0']
    print(val)
    lr=LogisticRegression(featuresCol="w2v",labelCol=val,maxIter=800)
    lr_model=lr.fit(result)
    prediction=lr_model.transform(log_result)
    log_result=prediction.withColumnRenamed("prediction",val)
    log_result=log_result.drop(*["rawPrediction","probability"])
    log_result=log_result.withColumn(val,log_result[val].cast(IntegerType()))
    

Drama
Comedy
Romance Film
Thriller
Action
World cinema
Crime Fiction
Horror
Black-and-white
Indie
Action/Adventure
Adventure
Family Film
Short Film
Romantic drama
Animation
Musical
Science Fiction
Mystery
Romantic comedy


In [19]:
log_result=log_result.withColumn("predictions",lit(""))
for row in mapping_df.rdd.collect():
    val=row['0']
    log_result=log_result.withColumn("predictions",concat(col("predictions"),lit(" "),col(val)))

In [20]:
out=log_result.select("movie_id","predictions")
out.write.csv("sample_w2v.csv",mode="overwrite",header=True)